Hi, Now I am going to create a model for English to Tamil translation from scrach. Here I am going to use LSTM Model with the concept of Encoder, Decoder and Attention Model. Let have a look below.

Data Preprocessing: Tokenize, pad, and create vocabulary.

Encoder-Decoder Model:

Encoder: Processes English input into a context vector.

Decoder: Uses the context vector to generate Tamil output word-by-word.

Embedding Layer: Converts words into dense vectors for better understanding.

LSTM: Handles the sequence data for both encoder and decoder.

Attention Mechanism (Optional): Helps focus on specific parts of the input sentence during decoding.



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive

In [ ]:
drive.mount('/content/driven')

Mounted at /content/driven


In [ ]:
English = open('/content/driven/MyDrive/Translation data-Eng-Tam/train.eng_Latn','r')
Tamil = open('/content/driven/MyDrive/Translation data-Eng-Tam/train.tam_Taml','r')

In [ ]:
data = pd.DataFrame({'English': English,
    'Tamil': Tamil
})

In [ ]:
data

,English,Tamil
0,Jens Lehmann became the first player to be sen...,சாமுவேல் எட்டோவை வீழ்த்தியதன் மூலம் ஜென்ஸ் லெஹ...
1,Nigeria's best chance of the game was also in ...,நைஜீரிய அணிக்கும் கோலை அடிப்பதற்கான சிறந்த வாய...
2,Williams has previously represented the Wales ...,வில்லியம்ஸ் முன்பு 21 வயதுக்கு உட்பட்டோருக்கான...
3,"Further in the match, Carvajal was yellow card...","மேலும் இந்தப் போட்டியில், கர்வஜலுக்கு 83வது நி..."
4,"Yesterday, the German football club FC Bayern ...","நேற்று, ஜெர்மன் கால்பந்து கிளப் எஃப்சி பேயர்ன்..."
...,...,...
66964,He is named on the Philadelphia Eagles Honor R...,பிலடெல்ஃபியா ஈகிள்ஸ் மதிப்பு பட்டியலில் அவர் ப...
66965,He played college football at the University o...,அவர் மிச்சிகன் பல்கலைக் கழகத்தில் நடைபெற்ற கல்...
66966,"Sulon played for RFC Liège, UR Namur, RES Jamb...","ஆர்எஃப்சி லீக், யூஆர் நமூர், ஆர்ஈஎஸ் ஜம்போய்ஸ்..."
66967,He plays for Austrian Bundesliga club FC Red B...,அவர் ஆஸ்திரிய புண்டஸ்லிகா எஃப்சி ரெட் புல் சால...


In [ ]:
data['English'] = data['English'].iloc[:10000]
data['Tamil'] = data['Tamil'].iloc[:10000]

In [ ]:
data.dropna(inplace=True)

In [ ]:
data

,English,Tamil
0,Jens Lehmann became the first player to be sen...,சாமுவேல் எட்டோவை வீழ்த்தியதன் மூலம் ஜென்ஸ் லெஹ...
1,Nigeria's best chance of the game was also in ...,நைஜீரிய அணிக்கும் கோலை அடிப்பதற்கான சிறந்த வாய...
2,Williams has previously represented the Wales ...,வில்லியம்ஸ் முன்பு 21 வயதுக்கு உட்பட்டோருக்கான...
3,"Further in the match, Carvajal was yellow card...","மேலும் இந்தப் போட்டியில், கர்வஜலுக்கு 83வது நி..."
4,"Yesterday, the German football club FC Bayern ...","நேற்று, ஜெர்மன் கால்பந்து கிளப் எஃப்சி பேயர்ன்..."
...,...,...
1995,Thrashers goaltender Kari Lethonen was almost ...,த்ராஷர்ஸ் அணியின் கோல்டெண்டர் காரி லெத்தோனென் ...
1996,"In the final period, the Penguins rallied back...","இறுதி கட்டத்தில், பென்குயின்ஸ் அணி பவர் பிளேயி..."
1997,"Backed by Thibault's strong play in net, Pengu...",வலையில் திபோல்ட்டின் வலுவான விளையாட்டின் ஆதரவு...
1998,"In the second half of the 20th century, Sattri...","20ஆம் நூற்றாண்டின் இரண்டாம் பாதியில், சத்ரிய ந..."


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data['English'].shape

(6000,)

In [ ]:
len(data['Tamil'])

6000

In [ ]:
data['Tamil'][0]

'<start> சாமுவேல் எட்டோவை வீழ்த்தியதன் மூலம் ஜென்ஸ் லெஹ்மன் இறுதிப் போட்டியில் வெளியேற்றப்பட்ட முதல் வீரரானார்.\n <end>'

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding


In [ ]:
english_tokenizer = Tokenizer()
tamil_tokenizer = Tokenizer()

data['Tamil'] = data['Tamil'].apply(lambda x: '<start> ' + x + ' <end>')


In [ ]:
english_tokenizer.fit_on_texts(data['English'])
tamil_tokenizer.fit_on_texts(data['Tamil'])

english_sequences = english_tokenizer.texts_to_sequences(data['English'])
tamil_sequences = tamil_tokenizer.texts_to_sequences(data['Tamil'])

In [ ]:
max_english_length = max([len(seq) for seq in english_sequences])
max_tamil_length = max([len(seq) for seq in tamil_sequences])

english_padded = pad_sequences(english_sequences, maxlen=max_english_length, padding='post')
tamil_padded = pad_sequences(tamil_sequences, maxlen=max_tamil_length, padding='post')

In [ ]:
# Encoder Model
encoder_inputs = Input(shape=(max_english_length,))
encoder_embedding = Embedding(input_dim=len(english_tokenizer.word_index)+1, output_dim=128)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [ ]:
# Decoder Model
decoder_inputs = Input(shape=(max_tamil_length - 1,))
decoder_embedding = Embedding(input_dim=len(tamil_tokenizer.word_index)+1, output_dim=256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(tamil_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Seq2Seq Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(
    [english_padded, tamil_padded[:, :-1]],
    tamil_padded[:, 1:],
    batch_size=64,
    epochs=30,
    validation_split=0.2
)

In [ ]:
# Encoder Model for Inference
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
# Decoder Model for Inference
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embedding_inference = Embedding(input_dim=len(tamil_tokenizer.word_index)+1, output_dim=256)(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding_inference, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
# Translation Function
def translate_sentence(input_text):
    # Tokenize and pad the input English sentence
    input_seq = english_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_english_length, padding='post')

    # Get the encoder states
    states_value = encoder_model.predict(input_seq)

    # Initialize the target sequence with the <start> token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tamil_tokenizer.word_index.get('<start>', 1)

    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        # Predict the next token in the sequence
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the most probable next token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tamil_tokenizer.index_word.get(sampled_token_index, '')

        # Stop if we reach the <end> token or exceed the max length
        if sampled_word == '<end>' or len(decoded_sentence) > max_tamil_length:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)

        # Update target sequence and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return " ".join(decoded_sentence).strip()

In [ ]:
# Example sentence for translation
test_sentence = "Jens Lehmann became the first player to be"
translated_sentence = translate_sentence(test_sentence)

print("English:", test_sentence)
print("Tamil:", translated_sentence)


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1447s 14s/step - accuracy: 0.7129 - loss: 5.3450 - val_accuracy: 0.8011 - val_loss: 2.0391
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1460s 14s/step - accuracy: 0.7824 - loss: 2.0507 - val_accuracy: 0.8015 - val_loss: 2.0299
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1464s 14s/step - accuracy: 0.7836 - loss: 1.9884 - val_accuracy: 0.8021 - val_loss: 2.0297
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1455s 14s/step - accuracy: 0.7838 - loss: 1.9584 - val_accuracy: 0.8022 - val_loss: 2.0433
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1446s 14s/step - accuracy: 0.7828 - loss: 1.9331 - val_accuracy: 0.8025 - val_loss: 2.0548
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1453s 14s/step - accuracy: 0.7852 - loss: 1.8830 - val_accuracy: 0.8021 - val_loss: 2.0754
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1465s 14s/step - accuracy: 0.7867 - loss: 1.8401 - val_accuracy: 0.8015 - val_loss: 2.0965
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1445s 14s/step - accuracy: 0.7853 - loss: 1

In [ ]:
# Example sentence for translation
test_sentence = "player"
translated_sentence = translate_sentence(test_sentence)

print("English:", test_sentence)
print("Tamil:", translated_sentence)


1/1 [==============================] - 0s 28ms/step
English: player
Tamil: இந்த ஒரு ஒரு ஒரு ஒரு ஒரு ஒரு ஒரு ஒரு ஒரு ஒரு ஒரு கோல் அடித்தார் end end end end end end end end end end end end end end end end end


In [ ]:
model.save('my_model.keras')

In [ ]:
model.save("Finalh5.h5") # Change the file extension to '.keras'

In [ ]:
joblib.load('my_model_joblib')

In [ ]:
import pickle

# Save the model as a pickle file
with open("model_pkl.pkl", "wb") as file:
    pickle.dump(model, file)
